let's start off by getting the libraries for our webscraping and for word2vec

In [1]:
from selenium.webdriver import Chrome
import time
import random
import json
import pandas as pd
import string
import re
import gensim
import numpy as np
import functions as fc

In [ ]:
fc.

set up for our web scraper and ease of access to a web blocker to install on the selenium instance

In [20]:
# browser = Chrome()


# urls = ["https://www.allrecipes.com/recipes/76/appetizers-and-snacks/",
#         "https://www.allrecipes.com/recipes/156/bread/",
#         "https://www.allrecipes.com/recipes/78/breakfast-and-brunch/",
#         "https://www.allrecipes.com/recipes/276/desserts/cakes/",
#         "https://www.allrecipes.com/recipes/201/meat-and-poultry/chicken/",
#         "https://www.allrecipes.com/recipes/362/desserts/cookies/",
#         "https://www.allrecipes.com/recipes/430/seafood/shellfish/shrimp/",
#         "https://www.allrecipes.com/recipes/253/everyday-cooking/slow-cooker/",
#         "https://www.allrecipes.com/recipes/1227/everyday-cooking/vegan/",
#         "https://www.allrecipes.com/recipes/86/world-cuisine/"]

# browser.set_page_load_timeout(300)
# browser.get('https://chrome.google.com/webstore/detail/adblock/gighmmpiobklfepjocnamgkkbiglidom')

let the scraping commence!

In [2]:
# for url in urls:
#     print(url)
#     browser.get(url)

#     for _ in range(6):
#         browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(3 + random.random()*10)

#     link_list = browser.find_elements_by_xpath('//*[contains(@class,"fixed-recipe-card__info")]/a')

#     recipe_page_list = [link.get_attribute('href') for link in link_list]

#     len(recipe_page_list)

#     with open('recipes_list.json', 'a') as f:


#         for recipe in recipe_page_list:

#             browser.get(recipe)

#             recipe_dict = {}
            
#             try:
#                 recipe_dict['title'] = browser.find_element_by_xpath('//*[@id="recipe-main-content"]').text
#             except:
#                 recipe_dict['title'] = browser.find_element_by_xpath('//*[contains(@class, "intro article-info")]/h1').text
            
#             try:
#                 lala_list = browser.find_elements_by_xpath('//*[contains(@id,"lst_ingred")]')
#                 item_list = []
#                 for element in lala_list:

#                     item_list += element.find_elements_by_css_selector('span')

#                 ingredients = [item.text for item in item_list[:-1]]
#                 recipe_dict['ingredients'] = ingredients
            
#             except:
#                 lala_list = browser.find_elements_by_xpath('//*[contains(@class, "ingredients-item-name")]')

#                 ingredients = [item.text for item in lala_list]
#                 recipe_dict['ingredients'] = ingredients

#             json.dump(recipe_dict, f)
#             f.write('\n')
#             time.sleep(3 + random.random()*10)

In [3]:
#bring in the json data line by line
data = []
with open('recipes_list.json') as file:
    for line in file:
        data.append(json.loads(line))


In [4]:
df = pd.DataFrame(data)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 2 columns):
ingredients    929 non-null object
title          929 non-null object
dtypes: object(2)
memory usage: 14.6+ KB


In [6]:

indices_list = []
for i in range(len(df.ingredients)):
    
    
        try: 
            if df.ingredients[i][0] == '':
                print(df.ingredients[i])
                indices_list.append(i)
                
        except IndexError:
            print(df.ingredients[i])
            indices_list.append(i)


print(indices_list)

[]
['', '', '', '', '', '', '', '', '', '']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[60, 136, 217, 220, 222, 251, 436, 441, 509, 710, 713, 731, 778, 788, 867]


In [7]:
to_be_reindexed_df = df.drop(indices_list)

clean_df = to_be_reindexed_df.reset_index(drop=True)
clean_df.head()

,ingredients,title
0,"[2 pounds unpeeled large shrimp, 2 tablespoons...",Grilled Shrimp Scampi Cocktail
1,"[cooking spray, 24 (1 inch) cubes fresh pineap...",Easy Rumaki with Pineapple
2,"[2 tablespoons butter, 2 tablespoons minced gr...",Perfect Crab-Stuffed Mushrooms
3,"[3 cups diced cooked rotisserie chicken, 2 (8 ...",Baked Buffalo Chicken Dip
4,"[24 slices French baguette, 1 tablespoon butte...",Strawberry Bruschetta


In [8]:
clean_df.ingredients[0]

['2 pounds unpeeled large shrimp',
 '2 tablespoons olive oil',
 '1 teaspoon minced garlic',
 '1/4 teaspoon salt',
 '1/8 teaspoon freshly ground black pepper',
 '1 cup cocktail sauce',
 '1 lemon, cut into wedges']

In [45]:
def split_words_in_str_lst_items(string_list):
    """Splits a list of sentences into a
    list with lists of individual words as values"""
    
    split_words = []
    for item in string_list:
        split_words.append(item.split())
    return split_words
        

def lst_clean_cooking_stop_words(string_list):
    """Remove measurement and cooking words,
    leaving only ingredients"""
    
    cooking_stop_words = [
    'canned', 'cans', 'drained', 'and', 'halved', 'cup', 'cups',
    'teaspoon', 'tablespoon', 'teaspoons', 'tablespoons',
    'finely', 'freshly', 'fresh', 'thickcut', 'to', 'taste',
    'grated', 'cut', 'into', 'wedges', 'pounds', 'unpeeled', 'large',
    'minced', 'slice', 'slices', 'sliced', 'thick-cut', 'cut', 'crosswise', 'pieces',
    'toothpicks', 'low-fat', 'chopped', 'or', 'taste', 'cooked', 'dry',
    'shredded', 'beaten', 'dried', 'melted', 'stems', 'removed', 'diced',
    'ounce', 'ounces', 'packages', 'softened', 'such', 'RedHot®', 'RedHot', 'Franks', "Frank's",
    'crumbled', 'Old', 'Bay®', 'Bay', 'pinch', 'for', 'garnish', 'slice', 'slices',
    'needed', 'inch', 'cubes', 'cooking', 'spray', 'ground', 'rotisserie',
    'lowfat', 'as', 'quarteres', 'cloves', 'more', 'can', 'package', 'frozen',
    'thawed', 'packet', 'reducedfat', 'Knorr', 'container', 'pound', 'peeled',
    'deveined', 'seeded', 'ripe', 'English', 'juiced', 'plus', 'more', 'Hass',
    'cubed', 'Mexicanstyle', 'hearts', 'prepared', 'party', 'pitted', 'mashed',
    'roma', 'optional', 'chunk', 'Hot', 'bunch', 'cleaned', 'box', 'chickenflavored',
    'Golden', 'delicious', 'cored', 'any', 'flavor', 'flavored', 'whole', 'allpurpose',
    'all', 'purpose', 'deep', 'frying', 'dash', 'packed', 'in', 'French', 'jar',
    'small', 'head', 'little', 'smokie', 'seasoned', 'Boston', 'Bibb', 'leaves',
    'lean', 'pickled', 'Asian', 'dark', 'flaked', 'rolled', 'packed', 'jellied',
    'thirds', 'with', 'attached', 'skewers', 'skinless', 'boneless', 'half', 'kernels',
    'rinsed', 'quart', 'quarts', 'kernel', 'Italianstyle', 'unpopped', 'lightly',
    'coating', 'SAUCE', 'lengthwise', 'miniature', 'semisweet', 'rinsed', 'round',
    'squeezed', 'stewed', 'raw', 'the', 'liquid', 'reserved', 'medium', 'instant',
    'solid', 'pack', 'refrigerated', 'halves', 'distilled', 'loaf', 'extra', 'virgin',
    'crushed', 'kosher', 'toasted', 'buttery', 'TM', 'panko', 'Japanese', 'regular',
    'bottle', 'bottles', 'thin', 'peel', 'paper', 'thick', 'circles', 'unbleached',
    'breast', 'breasts', 'wings', 'strips', 'jumbo', 'giant', 'chunks', 'quickcooking',
    'sweetened', 'flakes', 'Ranchstyle', 'snipped', 'food', 'ROTEL', 'Italian', 'sticks',
    'stick', 'crescent', 'thinly', 'boiled', 'Genoa', 'roasted', 'thin', 'extrasharp',
    'pressed', 'sifted', 'split', 'tips', 'discarded', 'mini', 'deli', 'drain', 'reserve',
    'diameter', 'Greek', 'Thai', 'drops', 'square', 'crusty', 'American', 'selfrising',
    'imitation', 'Wings', 'apart', 'at', 'joints', 'wing', 'tips', 'discarded', 'parts',
    'tops', 'seperated', 'blend', 'coarsely', 'sweet', 'stalk', 'heads', 'husked',
    'divided', 'pats', 'unsalted', 'active', 'warm', 'sea', 'separated', 'herb',
    'overripe', 'degrees', 'F', 'C', 'room', 'temperature', 'machine', 'very',
    'pint', 'puree', 'coarse', 'envelopes', 'lukewarm', 'creamstyle', 'unsweetened',
    'lite', 'of', 'chilled', 'freezer', 'cold', 'brushing', 'nonfat', 'squares',
    'tails', 'thigh', 'quarters', 'Masterpiece', 'KC', 'from', 'El', 'Paso', 
    'bulk', 'Hunts', 'Roma', 'light', 'fluid', 'lagerstyle', 'stalks', 'quartered',
    'undrained', 'drained', 'Tony', 'Chacheres', 'lump', 'uncooked', 'cube',
    'bits', 'hair', 'angel', 'trimmed', 'stew', 'spaghetti', 'brisket',
    'bitesized', 'matchstick', 'Chobani', 'unbaked', 'crust', 'torn',
    'bonein', 'pounded', 'bitesize', 'granules', 'boiling', 'yolk',
    'coloring', 
    
    ]
    
    cleaned_items = []
    for item_string in string_list:
        for word in item_string:
            if word not in cooking_stop_words:

                cleaned_items.append(word)
    return cleaned_items

def lst_strip_numbers_and_punc(string_list):
    """Remove numbers and punctuation from a list of strings"""
    
    num_removed = []
    for x in string_list:
        current_string = re.sub("\d+", "", x)
        current_string = re.sub(r'[^\w\s]','',current_string)


        num_removed.append(current_string)
    return num_removed

def clean_ingredients(string_list):
    """Take in a list of strings, strip it of numbers and punctuation,
    then split it into individual words, finally remove non-ingredient words"""
    
    stage_1 = lst_strip_numbers_and_punc(string_list)
    stage_2 = split_words_in_str_lst_items(stage_1)
    stage_3 = lst_clean_cooking_stop_words(stage_2)
    return stage_3
    
    
    
def parse_ingred_tuples(tupp_list):
    """Pull the first value from a list of tuples"""
    
    sim_ingreds = []
    for i in range(len(tupp_list)):
        sim_ingreds.append(tupp_list[i][0])
    return sim_ingreds

def ingred_series_compare(ingredient_series, compare_list):
    """Check if each list in a series of lists contains specific words"""
    
    indices_of_recipes = []
    for k,j in enumerate(ingredient_series):
        for i in j:
            if i in compare_list:
                indices_of_recipes.append(k)
    return indices_of_recipes

def sound_tasty(word_vec_model):
    try:
        ingre_list = parse_ingred_tuples(word_vec_model.wv.most_similar(
            positive=[input('Choose an ingredient: ').lower(),
            input('Choose another: ').lower(),
            input('And another: ').lower(),
            input('Last one: ').lower()],
            topn=100000)
            )



        user_input = input("""How adventurous are you feeling? low, medium, or high? """)
        print('\n')



        if user_input == "low":
            comp_list = ingre_list[:20]
            print("I'll get you food similar to what you're used too")
        elif user_input == "medium":
            comp_list = ingre_list[round(len(ingre_list)/2):round(len(ingre_list)/2)+20]
            print("I'll get you food sort of like what you're used too")
        elif user_input == "high":
            comp_list = ingre_list[-20:]
            print("I'll get you food pretty different from what you're used too")
        else:
            print("that's not an option: defaulting to 'low' ")
            comp_list = ingre_list[:20]

        print('\n\n\n',
              'ingredients like yours:',
              '\n\n',
              comp_list)


        count_me = ingred_series_compare(clean_df.ingredients, comp_list)


        ingred_count_dict = {i: count_me.count(i) for i in count_me}

        high_value = (0,0)
        for x in ingred_count_dict.keys():
            if ingred_count_dict[x] > high_value[1]:
                high_value = (x, ingred_count_dict[x])

        runners_up = []

        for x in ingred_count_dict.keys():
            if ingred_count_dict[x] == high_value[1] or ingred_count_dict[x] == (high_value[1] - 1):
                if x is not high_value[0]:
                    runners_up.append((x, ingred_count_dict[x]))

        print('\n\n',
              "How about? ",
              "\n\n",
              clean_df.title[high_value[0]],
              '\n\n',
              clean_df.ingredients[high_value[0]],
              '\n\n\n\n')
        if input('Sound tasty? yes/no: ') == "yes":
            pass
        else:
            print('\n\n',
                  "Fine, I'll find you something else. You're welcome.",
                  '\n\n')
            if len(runners_up) == 0:
                print("I got nothing.")
            for i in range(len(runners_up)):
                print('\n\n',
                      "How about? "
                      '\n\n',
                      clean_df.title[runners_up[i][0]],
                      '\n\n',
                      clean_df.ingredients[runners_up[i][0]],
                      '\n\n\n\n')

                if input('Sound tasty? yes/no: ') == "yes":
                    print('\n\n', 'Go eat it then!')
                    break
                else:
                    print('\n\n')
                    choice = random.choice(["Not good enough hmm? let me try again.",
                                            "You're picky aren't you?",
                                            "It sounds like you already know what you want, but whatever.",
                                            "Are you serious? gosh... fine!",
                                            "Mmmm, you look tasty",
                                            "Any objections to eating brains?",
                                            "You could always order a pizza.",
                                            "I've heard interest rates are on the move again. Thoughts?",
                                            "Don't skip leg day.",
                                            "YOU CAN'T HANDLE THE TRUTH!",
                                            "Maybe a churro? Who doesn't love churros?",
                                            "I've got a dark and mysterious past.",
                                            "In another life, they called me... Tim"])
                    print(choice)      

    except KeyError:
        print('Ingredient not recognized... sorry :(')

In [9]:
clean_df.ingredients = clean_df.ingredients.map(fc.clean_ingredients)

In [10]:
model = gensim.models.Word2Vec(clean_df.ingredients, sg=1)

model.train(clean_df.ingredients,
            total_examples=model.corpus_count,
            epochs=model.epochs)

model.corpus_total_words

model.wv

In [11]:
fc.sound_tasty(model, dataframe=clean_df)

Choose an ingredient: cheese
Choose another: eggs
And another: bacon
Last one: sausage
How adventurous are you feeling? low, medium, or high? low


I'll get you food similar to what you're used too



 ingredients like yours: 

 ['halfandhalf', 'Swiss', 'cottage', 'mayonnaise', 'potato', 'Red', 'rolls', 'tortilla', 'heavy', 'biscuit', 'broccoli', 'ranch', 'provolone', 'ham', 'Romano', 'steak', 'crackers', 'crabmeat', 'Cheddar', 'sage']


 How about?  

 Antipasto Squares 

 ['dinner', 'rolls', 'ham', 'provolone', 'cheese', 'Swiss', 'cheese', 'salami', 'pepperoni', 'sausage', 'red', 'peppers', 'eggs', 'Parmesan', 'cheese', 'black', 'pepper'] 




Sound tasty? yes/no: yes
